In [0]:
import pandas as pd
import numpy as np
import keras
# from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold, train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
# from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import pickle
import time
import tensorflow as tf
from keras.datasets import cifar10
import cv2




In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
def f1(y_true, y_pred):
    y_pred = np.rint(y_pred)
    tp = sum(y_true*y_pred)
    tn = sum((1-y_true)*(1-y_pred))
    fp = sum((1-y_true)*y_pred)
    fn = sum(y_true*(1-y_pred))

    p = tp / (tp + fp )
    r = tp / (tp + fn )

    f1 = 2*p*r / (p+r)
    return f1

In [0]:
start_time = time.time()
y = pd.read_csv("/content/gdrive/My Drive/data/y_bush_vs_others.csv", header= None)
y_bush = y.values.ravel()
y = pd.read_csv("/content/gdrive/My Drive/data/y_williams_vs_others.csv", header= None)
y_williams = y.values.ravel()
X = pd.read_csv("/content/gdrive/My Drive/data/X.csv", sep=' ', header= None, dtype= float)
X = X.values
print("--- %s seconds ---" % (time.time() - start_time))

--- 23.979103803634644 seconds ---


In [0]:
X_train, X_test, y_williams_train, y_williams_test = train_test_split(X,y_williams, test_size=1./3, 
                                                                      random_state = 6926, shuffle = True, stratify = y_williams)

In [0]:
X_train = X_train.reshape(8822,64,64,1)
X_test = X_test.reshape(4411,64,64,1)
y_williams_train = y_williams_train.reshape(8822,1)
y_williams_test = y_williams_test.reshape(4411,1)

In [0]:
williams_model = Sequential()
williams_model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='relu',input_shape=(64,64,1)))
williams_model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='relu'))
williams_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

williams_model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
williams_model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
williams_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

williams_model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
williams_model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
williams_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

williams_model.add(Flatten())
williams_model.add(Dense(1024,activation='relu'))
williams_model.add(Dense(1,activation='sigmoid'))
williams_model.compile(loss=keras.losses.binary_crossentropy, optimizer='adam', metrics=['accuracy'])

In [0]:
(xC_train, yC_train), (xC_test, yC_test) = cifar10.load_data()


In [0]:
# binary labels
for index, y in enumerate(yC_train):
    if not y==1:
        yC_train[index] = 0
        
        
        
for index, y in enumerate(yC_test):
    if not y==1:
        yC_test[index] = 0

In [0]:
xC_test.shape

(10000, 32, 32, 3)

In [0]:
# convert to grayscale 
def rgb2gray(rgb):

    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

In [0]:
# and resize to 64*64
xC_train_gray = [None] * 50000
xC_test_gray = [None] * 10000
for index, x in enumerate(xC_train):
  cur = rgb2gray(x)
  xC_train_gray[index] = cv2.resize(cur, dsize=(64,64), interpolation=cv2.INTER_CUBIC)
  
for index, x in enumerate(xC_test):
  cur = rgb2gray(x)
  xC_test_gray[index] = cv2.resize(cur, dsize=(64,64), interpolation=cv2.INTER_CUBIC)

In [0]:
xC_train_gray = np.array(xC_train_gray)
xC_test_gray = np.array(xC_test_gray)

xC_train_gray.shape

(50000, 64, 64)

In [0]:
# reshape it into 32*32*1
xC_train_gray = xC_train_gray.reshape(50000,64,64,1)
xC_test_gray = xC_test_gray.reshape(10000,64,64,1)

In [0]:
xC_test_gray.shape

(10000, 64, 64, 1)

In [0]:
yC_test.shape

(10000, 1)

In [0]:
xC_train_gray.shape

(50000, 64, 64, 1)

In [0]:
yC_train.shape

(50000, 1)

In [0]:
#pre-train the model with xC and yC
csv_logger = keras.callbacks.CSVLogger('/content/gdrive/My Drive/data/williams_training.log')
williams_model.fit(xC_train_gray, yC_train, batch_size = 2000, epochs = 10, verbose =1 ,callbacks=[csv_logger])
print(f1(y_williams_train,williams_model.predict(X_train)))
print(f1(y_williams_test,williams_model.predict(X_test)))

Epoch 1/10
50000/50000 [==============================] - 21s 413us/step - loss: 1.5681 - acc: 0.8926
Epoch 2/10
50000/50000 [==============================] - 20s 397us/step - loss: 1.6118 - acc: 0.9000
Epoch 3/10
50000/50000 [==============================] - 20s 398us/step - loss: 1.6118 - acc: 0.9000
Epoch 4/10
50000/50000 [==============================] - 20s 397us/step - loss: 1.6118 - acc: 0.9000
Epoch 5/10
46000/50000 [==========================>...] - ETA: 1s - loss: 1.6038 - acc: 0.9005

In [0]:
williams_model.save("/content/gdrive/My Drive/data/pretrain_williams_cifar.h5")

In [0]:
# then train the model on williams
csv_logger = keras.callbacks.CSVLogger('/content/gdrive/My Drive/data/williams_training_after.log')
williams_model.fit(X_train, y_williams_train, batch_size = 1000, epochs =20, verbose =1 ,callbacks=[csv_logger])
print(f1(y_williams_train,williams_model.predict(X_train)))
print(f1(y_williams_test,williams_model.predict(X_test)))

In [0]:
# train again
csv_logger = keras.callbacks.CSVLogger('/content/gdrive/My Drive/data/williams_training_after.log')
williams_model.fit(X_train, y_williams_train, batch_size = 1000, epochs = 20, verbose =1 ,callbacks=[csv_logger])
print(f1(y_williams_train,williams_model.predict(X_train)))
print(f1(y_williams_test,williams_model.predict(X_test)))

In [0]:
# train again
csv_logger = keras.callbacks.CSVLogger('/content/gdrive/My Drive/data/williams_training_after.log')
williams_model.fit(X_train, y_williams_train, batch_size = 1000, epochs = 20, verbose =1 ,callbacks=[csv_logger])
print(f1(y_williams_train,williams_model.predict(X_train)))
print(f1(y_williams_test,williams_model.predict(X_test)))

In [0]:
# train again
csv_logger = keras.callbacks.CSVLogger('/content/gdrive/My Drive/data/williams_training_after.log')
williams_model.fit(X_train, y_williams_train, batch_size = 2000, epochs = 20, verbose =1 ,callbacks=[csv_logger])
print(f1(y_williams_train,williams_model.predict(X_train)))
print(f1(y_williams_test,williams_model.predict(X_test)))

In [0]:
williams_model.save("/content/gdrive/My Drive/data/williams_cifar.model")

In [0]:
# train again
csv_logger = keras.callbacks.CSVLogger('/content/gdrive/My Drive/data/williams_training_after.log')
williams_model.fit(X_train, y_williams_train, batch_size = 2000, epochs = 20, verbose =1 ,callbacks=[csv_logger])
print(f1(y_williams_train,williams_model.predict(X_train)))
print(f1(y_williams_test,williams_model.predict(X_test)))

In [0]:
williams_model.save("/content/gdrive/My Drive/data/williams_cifar_0.model")

In [0]:
# train again
csv_logger = keras.callbacks.CSVLogger('/content/gdrive/My Drive/data/williams_training_after.log')
williams_model.fit(X_train, y_williams_train, batch_size = 2000, epochs = 20, verbose =1 ,callbacks=[csv_logger])
print(f1(y_williams_train,williams_model.predict(X_train)))
print(f1(y_williams_test,williams_model.predict(X_test)))

In [0]:
williams_model.save("/content/gdrive/My Drive/data/williams_cifar_1.model")

In [0]:
# train again
csv_logger = keras.callbacks.CSVLogger('/content/gdrive/My Drive/data/williams_training_after.log')
williams_model.fit(X_train, y_williams_train, batch_size = 2000, epochs = 20, verbose =1 ,callbacks=[csv_logger])
print(f1(y_williams_train,williams_model.predict(X_train)))
print(f1(y_williams_test,williams_model.predict(X_test)))

In [0]:
williams_model.save("/content/gdrive/My Drive/data/williams_cifar_2.model")

In [0]:
# train again
csv_logger = keras.callbacks.CSVLogger('/content/gdrive/My Drive/data/williams_training_after.log')
williams_model.fit(X_train, y_williams_train, batch_size = 2000, epochs = 20, verbose =1 ,callbacks=[csv_logger])
print(f1(y_williams_train,williams_model.predict(X_train)))
print(f1(y_williams_test,williams_model.predict(X_test)))

In [0]:
williams_model.save("/content/gdrive/My Drive/data/williams_cifar_3.model")